# Mount Diablo Challenge: Data Collection

This notebook demonstrates the data collection process for the Mount Diablo Challenge race analysis.

## Steps:
1. Scrape race results from all years
2. Fetch historical weather data
3. Store data in SQLite database

In [ ]:
# Setup
import sys
import os

# Add parent directory to path
sys.path.insert(0, os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

from scraper import DiabloScraper
from weather import WeatherFetcher
from database import DiabloDatabase
import config

import pandas as pd
from datetime import datetime

## 1. Scrape Race Results

Scrape race results from the Mount Diablo Challenge website for all available years.

In [ ]:
# Initialize scraper
scraper = DiabloScraper()

# Available years
years = config.RACE_YEARS
print(f"Will scrape data for years: {years}")

In [ ]:
# Scrape a single year for demonstration (2025)
demo_year = 2025
results_2025 = scraper.scrape_year(demo_year)

print(f"\nScraped {len(results_2025)} results for {demo_year}")
print(f"\nFirst result:")
print(results_2025[0])

In [ ]:
# Convert to DataFrame for easier viewing
df_2025 = pd.DataFrame(results_2025)
print(f"Data shape: {df_2025.shape}")
print(f"\nColumns: {df_2025.columns.tolist()}")
df_2025.head(10)

In [ ]:
# Uncomment to scrape ALL years (this will take several minutes)
# all_results = scraper.scrape_all_years(years)
# print(f"Total results scraped: {len(all_results)}")

## 2. Fetch Weather Data

Fetch historical weather data from Open-Meteo API for race dates.

In [ ]:
# Initialize weather fetcher
fetcher = WeatherFetcher()

# Get race date from 2025 results
race_date_2025 = results_2025[0]['race_date']
print(f"Race date for 2025: {race_date_2025}")

# Fetch weather for start location
weather_start = fetcher.fetch_weather_for_race(2025, race_date_2025, 'start')
print(f"\nFetched {len(weather_start)} weather records for START location")

# Fetch weather for summit location
weather_summit = fetcher.fetch_weather_for_race(2025, race_date_2025, 'summit')
print(f"Fetched {len(weather_summit)} weather records for SUMMIT location")

In [ ]:
# View weather data
df_weather = pd.DataFrame(weather_start + weather_summit)
print(f"\nWeather data shape: {df_weather.shape}")
df_weather

In [ ]:
# Calculate weather summary
summary = fetcher.get_weather_summary(weather_start + weather_summit)

print("\nWeather Summary for 2025:")
print("="*60)
for location, stats in summary.items():
    print(f"\n{location.upper()}:")
    for key, value in stats.items():
        if value is not None:
            print(f"  {key:20s}: {value:6.2f}")

## 3. Store Data in Database

Store the scraped data in SQLite database for persistent storage and analysis.

In [ ]:
# Initialize database
db = DiabloDatabase()
db.connect()
db.create_tables()

print("Database initialized")

In [ ]:
# Insert race results
db.insert_race_results(results_2025)
print(f"Inserted {len(results_2025)} race results")

In [ ]:
# Insert weather data
all_weather = weather_start + weather_summit
db.insert_weather_data(all_weather)
print(f"Inserted {len(all_weather)} weather records")

In [ ]:
# Check database contents
counts = db.get_table_counts()
print("\nDatabase contents:")
for table, count in counts.items():
    print(f"  {table}: {count} records")

In [ ]:
# Query data from database
query = "SELECT * FROM race_results WHERE year = 2025 LIMIT 10"
df_from_db = pd.read_sql_query(query, db.conn)

print("Sample data from database:")
df_from_db

In [ ]:
# Close database connection
db.close()
print("Database connection closed")

## Next Steps

- To collect data for all years, run the main pipeline script: `python3 main.py`
- For analysis, see notebook `02_analysis.ipynb`
- For visualizations, see notebook `03_visualization.ipynb`